## Data Collection by Web Scraping

In [1]:
from autoscraper import AutoScraper
import pandas as pd

In [46]:
recent=pd.DataFrame()
url='https://www.beeradvocate.com/beer/?view=recent'
scraper = AutoScraper()
brewery_name=['Drekker Brewing Company']
brewery_name = scraper.build(url, brewery_name)
review_profilename=['EMV']
review_profilename = scraper.build(url, review_profilename)
beer_style=['IPA - Black / Cascadian Dark Ale']
beer_style = scraper.build(url, beer_style)
beer_name=['Everybody Love Everybody']
beer_name = scraper.build(url, beer_name)

In [116]:
scraper = AutoScraper()
reviews=['look: 4 | smell: 4 | taste: 4.25 | feel: 4 | overall: 4']
reviews = scraper.build(url, reviews)
review_aroma=[]
review_appearance=[]
review_taste=[]
review_palate=[]
review_overall=[]
for i in reviews:
    i=list(i.split("|"))
    if len(i)<5:
        continue
    review_appearance.append(float(i[0].replace("look:","").strip()))
    review_aroma.append(float(i[1].replace("smell:","").strip()))
    review_taste.append(float(i[2].replace("taste:","").strip()))
    review_palate.append(float(i[3].replace("feel:","").strip()))
    review_overall.append(float(i[4].replace("overall:","").strip())) 

In [117]:
m=min(len(review_overall),len(brewery_name),len(review_profilename),len(beer_style),len(beer_name),len(review_appearance),len(review_aroma),len(review_taste),len(review_palate))
print(m)

25


In [118]:
recent=pd.DataFrame(list(zip(review_overall[:m],brewery_name[:m],review_profilename[:m],beer_style[:m],beer_name[:m],review_appearance[:m],review_aroma[:m],review_taste[:m],review_palate[:m])), columns=['review_overall','brewery_name','review_profilename','beer_style','beer_name','review_appearance','review_aroma','review_taste','review_palate'])
recent.head()

,review_overall,brewery_name,review_profilename,beer_style,beer_name,review_appearance,review_aroma,review_taste,review_palate
0,4.25,Drekker Brewing Company,EMV,Sour - Fruited Kettle Sour,Everybody Love Everybody,4.25,4.50,4.50,4.25
1,4.00,Bench Brewing Company,thehyperduck,Stout - American,Mountainview Stout,3.50,3.75,4.00,3.75
2,4.00,Artifex Brewing Company,mactrail,IPA - American,So Choice,3.75,3.25,3.50,3.50
3,4.75,Breckenridge Brewery,hreb,Winter Warmer,Christmas Ale,4.50,4.75,4.75,5.00
4,3.50,Fuller's,LanceBiggums,Old Ale,Vintage Ale,4.25,4.00,3.50,3.25


In [119]:
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as pl
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [120]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [121]:
beers_df = pd.read_csv(r"./BDA Miniproject/beer_reviews.csv" )

In [122]:
beers_df.shape

(1586614, 13)

In [123]:
beers_df=beers_df.drop(['brewery_id','review_time','beer_abv','beer_beerid'],axis=1)

In [124]:
beers_df.head()

,brewery_name,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name
0,Vecchio Birraio,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen
1,Vecchio Birraio,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon
2,Vecchio Birraio,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer
3,Vecchio Birraio,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils
4,Caldera Brewing Company,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA


In [125]:
beers_df=pd.concat([beers_df,recent])

In [126]:
beers_df.dtypes

brewery_name           object
review_overall        float64
review_aroma          float64
review_appearance     float64
review_profilename     object
beer_style             object
review_palate         float64
review_taste          float64
beer_name              object
dtype: object

In [127]:
beers_df.duplicated().sum()

822

In [128]:
beers_df.drop_duplicates(keep = 'first', inplace = True)

In [129]:
beers_df.duplicated().sum()
beers_df.head()

,brewery_name,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name
0,Vecchio Birraio,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen
1,Vecchio Birraio,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon
2,Vecchio Birraio,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer
3,Vecchio Birraio,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils
4,Caldera Brewing Company,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA


## Data Exploration

In [130]:
print( 'Unique breweries' )
print( 'By name:', beers_df[ 'brewery_name' ].nunique() )

Unique breweries
By name: 5759


In [131]:
print( 'Unique beers' )
print( 'By name:', beers_df[ 'beer_name' ].nunique() )

Unique beers
By name: 56876


In [132]:
print( 'Unique users:', beers_df[ 'review_profilename' ].nunique() )
print( 'Unique users with more than 1 review:', beers_df[ 'review_profilename' ].value_counts()[ beers_df[ 'review_profilename' ].value_counts() > 1 ].shape[ 0 ], '-' , str( round( beers_df[ 'review_profilename' ].value_counts()[ beers_df[ 'review_profilename' ].value_counts() > 1 ].shape[ 0 ] / beers_df[ 'review_profilename' ].nunique(), 2 ) * 100 ) + '%' )

Unique users: 33403
Unique users with more than 1 review: 22943 - 69.0%


In [133]:
beers_df[ 'review_profilename' ].value_counts().head()

northyorksammy    5814
BuckeyeNation     4661
mikesgroove       4612
Thorpe429         3518
womencantsail     3497
Name: review_profilename, dtype: int64

In [134]:
# A beer subset removing review information is created 
grouped_beers_df = beers_df[ [ 'beer_name', 'brewery_name', 'beer_style' ] ].drop_duplicates()

In [135]:
grouped_beers_df.shape

(65778, 3)

In [136]:
grouped_beers_df.loc[ grouped_beers_df.duplicated( subset = [ 'beer_name' ], keep = False ) ].sort_values( by = 'beer_name'  ).shape

(11483, 3)

In [137]:
# Beers by brewery
grouped_beers_df[ 'brewery_name' ].value_counts( dropna = False ).head()

Rock Bottom Restaurant & Brewery      869
John Harvard's Brewery & Ale House    394
Goose Island Beer Co.                 368
Iron Hill Brewery & Restaurant        316
Minneapolis Town Hall Brewery         312
Name: brewery_name, dtype: int64

In [138]:
# Beers by style
grouped_beers_df[ 'beer_style' ].value_counts( dropna = False ).head()

American IPA                3589
American Pale Ale (APA)     3320
American Amber / Red Ale    2489
American Porter             2181
Fruit / Vegetable Beer      1833
Name: beer_style, dtype: int64

In [139]:
# An new meassure is created by averaging review by factor
beers_df[ 'review_average' ] = round( ( ( beers_df[ 'review_overall' ] + beers_df[ 'review_aroma' ] + beers_df[ 'review_appearance' ] + beers_df[ 'review_palate' ] + beers_df[ 'review_taste' ] ) / 5 ) * 2 ) / 2

## General Filtering

In [140]:
# Defining the linear model
linear_model = LinearRegression( normalize = True )

In [141]:
# Training and generating predictions for the model
linear_model.fit( X = beers_df[ [ 'review_aroma', 'review_appearance', 'review_palate', 'review_taste' ] ], y = beers_df[ 'review_overall' ] )
preds = linear_model.predict( beers_df[ [ 'review_aroma', 'review_appearance', 'review_palate', 'review_taste' ] ] )

In [142]:
# Coeffifients for each feature (aroma, appearance, palate, taste)
linear_model.coef_

array([0.04781174, 0.03587682, 0.25846106, 0.55152083])

In terms of correlation, the ranking seems to be: taste, palate, aroma and appearance. 



These are the beer styles for the top 10 of beers with highest aroma and appearence reviews.

In [143]:
beers_df.sort_values( by = [ 'review_aroma', 'review_appearance' ], ascending = False ).head( 10 ) \
    [ 'beer_style' ].unique().tolist()

['Russian Imperial Stout',
 'American IPA',
 'American Porter',
 'American Double / Imperial IPA',
 'English Bitter',
 'American Pale Ale (APA)',
 'Scotch Ale / Wee Heavy',
 'Scottish Ale',
 'American Blonde Ale']

In [150]:
reviews_count_by_profilename = beers_df.groupby('review_profilename').review_overall.count().to_frame('count_reviews').sort_values(by = "count_reviews", ascending = False)
reviews_count_by_profilename.reset_index(inplace=True)
reviews_count_by_profilename.head()

,review_profilename,count_reviews
0,northyorksammy,5814
1,BuckeyeNation,4661
2,mikesgroove,4612
3,Thorpe429,3518
4,womencantsail,3497


In [151]:
reviews_count_user = reviews_count_by_profilename.count_reviews.value_counts().to_frame().reset_index()
reviews_count_user.columns = ['number_of_reviews','number_of_users']
reviews_count_user.head()

,number_of_reviews,number_of_users
0,1,10460
1,2,4106
2,3,2427
3,4,1599
4,5,1247


In [152]:
reviews_count_user['percentage_users'] = (reviews_count_user['number_of_users']*100)/reviews_count_user.number_of_users.sum()
reviews_count_user['cumul_percentage_users'] = reviews_count_user.percentage_users.cumsum()
reviews_count_user.head()

,number_of_reviews,number_of_users,percentage_users,cumul_percentage_users
0,1,10460,31.314553,31.314553
1,2,4106,12.292309,43.606862
2,3,2427,7.265814,50.872676
3,4,1599,4.786995,55.659671
4,5,1247,3.733198,59.392869


In [163]:
reviews_count_beer_id = beers_df.groupby('beer_name').review_overall.count().to_frame('Reviews_count').sort_values(by = "Reviews_count", ascending=False)
reviews_count_beer_id=reviews_count_beer_id.reset_index()

In [164]:
beer_ids_no_of_ratings_grt_30 = reviews_count_beer_id.loc[reviews_count_beer_id.Reviews_count>=30]['beer_name'].to_frame('beer_name')
users_no_of_ratings_grt_30 = reviews_count_by_profilename.loc[reviews_count_by_profilename.count_reviews>=30].review_profilename.to_frame('review_profilename')

In [167]:
filter_beers_df = pd.merge(beers_df, beer_ids_no_of_ratings_grt_30, how='inner', on='beer_name')
filter_beers_df = pd.merge(rating2, users_no_of_ratings_grt_30, how='inner', on='review_profilename')

## Recommendation Model 

In [168]:
import sklearn
from sklearn.model_selection import train_test_split

In [182]:
train, test = train_test_split(filter_beers_df, test_size=0.32, random_state=31)

In [183]:
print(train.shape)
print(test.shape)

(843333, 10)
(396864, 10)


In [184]:
train.reset_index(inplace=True,drop=True)
test.reset_index(inplace=True,drop=True)
train.head()

,brewery_name,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,review_average
0,Flying Dog Brewery,4.0,3.5,3.0,joepais,English Barleywine,3.5,3.5,Horn Dog Barley Wine Style Ale,3.5
1,Tröegs Brewing Company,4.0,4.0,4.0,biggred1,American Amber / Red Ale,4.0,4.0,Tröegs Hopback Amber Ale,4.0
2,Deschutes Brewery,4.0,3.5,3.5,GilGarp,American Blonde Ale,4.0,3.5,Twilight Ale,3.5
3,AleSmith Brewing Company,4.5,4.5,5.0,ehammond1,American Double / Imperial Stout,4.5,4.5,AleSmith Speedway Stout - Barrel Aged,4.5
4,Carlow Brewing Company,4.0,4.0,4.5,yelterdow,Irish Dry Stout,3.5,3.5,O'Hara's Irish Stout,4.0


In [173]:
df_beer_features = train.pivot_table(index='review_profilename',columns='beer_name',values='review_overall').fillna(0)

In [187]:
df_beer_features.head()

beer_name,"""Old Yeltsin"" Imperial Stout","""Shabadoo"" Black & Tan Ale",# 100,#'s Ale,#9,'t Gaverhopke Extra,'t Smisje BBBourgondier,'t Smisje Blond,'t Smisje Calva Reserva,'t Smisje Dubbel,'t Smisje Grande Reserva,'t Smisje IPA+,'t Smisje Kerst,(512) IPA,(512) Pecan Porter,(512) Whiskey Barrel Aged Double Pecan Porter,.357 Imperial Pilsner,10 Commandments,10 Degrees Below,10 Squared Fish Tale Ale,10.0,100 Meter Ale,100% Barrel Fermented Autumn Maple,1000 IBU,10th Anniversary Ale,10th Anniversary Double India Pale Ale,110K+OT Batch #2 - I.R.I.S.,110K+OT Batch #2 - I.R.I.S. - Barrel Aged,110K+OT Batch #3 - The Other West Coast IPA,110K+OT Batch #4 - Imperial Red Ale,11th Anniversary Double Wheat,11th Hour IPA,12 Dogs Of Christmas Ale,120 Minute IPA,12th Anniversary Undercover Investigation Shut-Down Ale,13 Rebels ESB,14'ER ESB,1516 Bavarian Lager,1554 Enlightened Black Ale,15th Anniversary Double Wench,15th Anniversary Wood Aged,16th Anniversary Wood Aged,17th Anniversary Wood Aged Double IPA,1800 Old English IPA,1837.0,1855 Celebration Lager,1888 Bock,18th Anniversary Pilsner,1900 Amber Lager,1919 - Choc Beer,19th Anniversary Belgian Style Golden Ale,2 Turtle Doves,200 Meter Ale,2006 Double Bastard Ale Aged In Brandy Barrels,2007 Vintage Oak Aged Ale (Barrel No. 230),2008 Vintage Oak Aged Ale (Barrel No. 96),2009 Correction Ale,2010 Christmas Ale,2012 Black Ale,20th Anniversary Pilsner,20th Anniversary Wit,21 IPA,21st Amendment IPA,21st Anniversary Belgian Strong Ale Aged With Old Vine Zin Grapes,21st Anniversary Fresh Hop Pale Ale,22nd Anniversary Vanilla Imperial Stout,25th Anniversary Vanilla Doubledog,2XIPA,2nd Anniversary Double IPA,2° Below Winter Ale,3 French Hens,3 Horses Holland Lager Beer,3 Judges Barleywine,3 Monts,3 Monts Grande Réserve Spécial Ale,30th Anniversary - Fritz And Ken's Ale,30th Anniversary - Grand Cru,30th Anniversary - Jack & Ken's Ale,"30th Anniversary- Charlie, Fred And Ken's Bock",30th Street Pale Ale,312 Urban Wheat,32/50,33 Export,3767 - Belgian IPA With Brett,3rd Anniversary Double IPA,4 Calling Birds,4 Seasons IPA,400 Pound Monkey,420 India Pale Ale,422 Pale Wheat Ale,46'er Pale Ale,471 ESB,471 IPA,483 Pale Ale,4th Anniversary Double IPA,5 A.M. Saint,5 Barrel Pale Ale,5 Day IPA,5 Years Later,5th Anniversary,5th Anniversary Ale,60 Minute IPA,60 Winks,606 Ale,6288 Stout,668 The Neighbor Of The Beast,6X,75 Minute IPA,77 Lager,8 Ball Stout,8-4-1 Expedition,80/- (Eighty Shilling),84/09 Double Alt (25th Anniversary Beer),840 India Pale Ale,8th Street Ale,90 Minute IPA,? The Riddler ?,A Little Sumpin' Extra! Ale,A Little Sumpin' Sumpin' Ale,A Little Sumpin' Wild,A Saison Darkly (Import Series Vol. #2),A-10 Warthog Double IPA,ACME California Brown Ale,ACME California IPA,ACME California Pale Ale,Aardmonnik - Earthmonk,Aass Bock,Abandon Ship Smoked Lager,Abbaye De Floreffe Double,Abbaye De Floreffe Prima Melior,Abbaye De Floreffe Triple,Abbaye De Saint Bon-Chien,Abbaye De Saint Bon-Chien Grand Cru (Aged In Trousseau Barrels),Abbaye Des Rocs Grand Cru Belgian Special Brown,Abbaye Des Rocs Spéciale Noël,Abbaye Des Rocs [Brasserie Des Rocs Brune],Abbey Ale,Abbey Belgian Style Ale,Abbey Dubbel,Abbey Six,Abbey Triple,Abbot Ale,Abner Imperial IPA,Abominable Winter Ale,Abrasive Ale,Acai Berry Wheat,Ace Of Spades,Acer Quercus - The Bruery / Lawson's Finest Liquids Collaboration,Aces & Ates,Achel Trappist Extra,Ad Astra Ale,Adam,Adam From The Wood,Adelscott,Adnams Bitter,Adnams Broadside,Adnams Broadside Strong Original,Adnams SSB (Suffolk Special Bitter),Adriaen Brouwer Dark Gold Ale,Aecht Schlenkerla Eiche,Aecht Schlenkerla Fastenbier,Aecht Schlenkerla Helles Lagerbier,Aecht Schlenkerla Rauchbier Märzen,Aecht Schlenkerla Rauchbier Urbock,Aecht Schlenkerla Rauchbier Weizen,Affligem Blond,Affligem Dubbel,Affligem Noël,Affligem Tripel,African Amber,Aftermath Imperial IPA,Afternoon Delight,Agave Wheat,Akevitt Porter (Barrel-Aged Porter),Alamo Golden Ale,Alaskan Amber,Alaskan Baltic Porter (Pilot Series),Alaskan

In [185]:
dummy_train = train.copy()
dummy_test = test.copy()

In [186]:
dummy_train['review_overall'] = dummy_train['review_overall'].apply(lambda x: 0 if x>=1 else 1)
dummy_test['review_overall'] = dummy_test['review_overall'].apply(lambda x: 1 if x>=1 else 0)

In [190]:
dummy_train = dummy_train.pivot_table(
    index='review_profilename',
    columns='beer_name',
    values='review_overall'
).fillna(1)

dummy_test = dummy_test.pivot_table(
    index='review_profilename',
    columns='beer_name',
    values='review_overall'
).fillna(0)
dummy_train.head()

beer_name,"""Old Yeltsin"" Imperial Stout","""Shabadoo"" Black & Tan Ale",# 100,#'s Ale,#9,'t Gaverhopke Extra,'t Smisje BBBourgondier,'t Smisje Blond,'t Smisje Calva Reserva,'t Smisje Dubbel,'t Smisje Grande Reserva,'t Smisje IPA+,'t Smisje Kerst,(512) IPA,(512) Pecan Porter,(512) Whiskey Barrel Aged Double Pecan Porter,.357 Imperial Pilsner,10 Commandments,10 Degrees Below,10 Squared Fish Tale Ale,10.0,100 Meter Ale,100% Barrel Fermented Autumn Maple,1000 IBU,10th Anniversary Ale,10th Anniversary Double India Pale Ale,110K+OT Batch #2 - I.R.I.S.,110K+OT Batch #2 - I.R.I.S. - Barrel Aged,110K+OT Batch #3 - The Other West Coast IPA,110K+OT Batch #4 - Imperial Red Ale,11th Anniversary Double Wheat,11th Hour IPA,12 Dogs Of Christmas Ale,120 Minute IPA,12th Anniversary Undercover Investigation Shut-Down Ale,13 Rebels ESB,14'ER ESB,1516 Bavarian Lager,1554 Enlightened Black Ale,15th Anniversary Double Wench,15th Anniversary Wood Aged,16th Anniversary Wood Aged,17th Anniversary Wood Aged Double IPA,1800 Old English IPA,1837.0,1855 Celebration Lager,1888 Bock,18th Anniversary Pilsner,1900 Amber Lager,1919 - Choc Beer,19th Anniversary Belgian Style Golden Ale,2 Turtle Doves,200 Meter Ale,2006 Double Bastard Ale Aged In Brandy Barrels,2007 Vintage Oak Aged Ale (Barrel No. 230),2008 Vintage Oak Aged Ale (Barrel No. 96),2009 Correction Ale,2010 Christmas Ale,2012 Black Ale,20th Anniversary Pilsner,20th Anniversary Wit,21 IPA,21st Amendment IPA,21st Anniversary Belgian Strong Ale Aged With Old Vine Zin Grapes,21st Anniversary Fresh Hop Pale Ale,22nd Anniversary Vanilla Imperial Stout,25th Anniversary Vanilla Doubledog,2XIPA,2nd Anniversary Double IPA,2° Below Winter Ale,3 French Hens,3 Horses Holland Lager Beer,3 Judges Barleywine,3 Monts,3 Monts Grande Réserve Spécial Ale,30th Anniversary - Fritz And Ken's Ale,30th Anniversary - Grand Cru,30th Anniversary - Jack & Ken's Ale,"30th Anniversary- Charlie, Fred And Ken's Bock",30th Street Pale Ale,312 Urban Wheat,32/50,33 Export,3767 - Belgian IPA With Brett,3rd Anniversary Double IPA,4 Calling Birds,4 Seasons IPA,400 Pound Monkey,420 India Pale Ale,422 Pale Wheat Ale,46'er Pale Ale,471 ESB,471 IPA,483 Pale Ale,4th Anniversary Double IPA,5 A.M. Saint,5 Barrel Pale Ale,5 Day IPA,5 Years Later,5th Anniversary,5th Anniversary Ale,60 Minute IPA,60 Winks,606 Ale,6288 Stout,668 The Neighbor Of The Beast,6X,75 Minute IPA,77 Lager,8 Ball Stout,8-4-1 Expedition,80/- (Eighty Shilling),84/09 Double Alt (25th Anniversary Beer),840 India Pale Ale,8th Street Ale,90 Minute IPA,? The Riddler ?,A Little Sumpin' Extra! Ale,A Little Sumpin' Sumpin' Ale,A Little Sumpin' Wild,A Saison Darkly (Import Series Vol. #2),A-10 Warthog Double IPA,ACME California Brown Ale,ACME California IPA,ACME California Pale Ale,Aardmonnik - Earthmonk,Aass Bock,Abandon Ship Smoked Lager,Abbaye De Floreffe Double,Abbaye De Floreffe Prima Melior,Abbaye De Floreffe Triple,Abbaye De Saint Bon-Chien,Abbaye De Saint Bon-Chien Grand Cru (Aged In Trousseau Barrels),Abbaye Des Rocs Grand Cru Belgian Special Brown,Abbaye Des Rocs Spéciale Noël,Abbaye Des Rocs [Brasserie Des Rocs Brune],Abbey Ale,Abbey Belgian Style Ale,Abbey Dubbel,Abbey Six,Abbey Triple,Abbot Ale,Abner Imperial IPA,Abominable Winter Ale,Abrasive Ale,Acai Berry Wheat,Ace Of Spades,Acer Quercus - The Bruery / Lawson's Finest Liquids Collaboration,Aces & Ates,Achel Trappist Extra,Ad Astra Ale,Adam,Adam From The Wood,Adelscott,Adnams Bitter,Adnams Broadside,Adnams Broadside Strong Original,Adnams SSB (Suffolk Special Bitter),Adriaen Brouwer Dark Gold Ale,Aecht Schlenkerla Eiche,Aecht Schlenkerla Fastenbier,Aecht Schlenkerla Helles Lagerbier,Aecht Schlenkerla Rauchbier Märzen,Aecht Schlenkerla Rauchbier Urbock,Aecht Schlenkerla Rauchbier Weizen,Affligem Blond,Affligem Dubbel,Affligem Noël,Affligem Tripel,African Amber,Aftermath Imperial IPA,Afternoon Delight,Agave Wheat,Akevitt Porter (Barrel-Aged Porter),Alamo Golden Ale,Alaskan Amber,Alaskan Baltic Porter (Pilot Series),Alaskan

## User Based Model Using Cosine Similarity

In [174]:
from sklearn.metrics.pairwise import pairwise_distances


user_correlation = 1 - pairwise_distances(df_beer_features, metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0

In [175]:
print(user_correlation)

[[1.         0.10668224 0.05271721 ... 0.04844577 0.02213635 0.00714799]
 [0.10668224 1.         0.0654051  ... 0.05567607 0.         0.        ]
 [0.05271721 0.0654051  1.         ... 0.1240652  0.03328539 0.04912214]
 ...
 [0.04844577 0.05567607 0.1240652  ... 1.         0.10047447 0.        ]
 [0.02213635 0.         0.03328539 ... 0.10047447 1.         0.07277543]
 [0.00714799 0.         0.04912214 ... 0.         0.07277543 1.        ]]


In [181]:
user_predicted_ratings = np.dot(user_correlation, df_beer_features.fillna(0))
user_predicted_ratings

array([[ 12.49693785,  19.91421464,  58.15031383, ..., 124.31024133,
         48.28565886,  66.00398164],
       [ 10.53800901,  23.97337772,  54.48312751, ...,  97.02168729,
         46.28361088,  63.15075922],
       [  9.50256632,  20.26892851,  40.09699798, ...,  60.72013102,
         47.46710352,  49.33095048],
       ...,
       [ 10.42889707,  22.05038107,  36.12855045, ...,  60.12861194,
         52.69367117,  46.94870025],
       [ 11.32504243,  21.74833149,  35.73814527, ...,  55.48440881,
         49.25933205,  39.67876557],
       [  6.99840896,  17.80788338,  25.48229588, ...,  39.06348696,
         39.63626531,  36.11485885]])

In [191]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

beer_name,"""Old Yeltsin"" Imperial Stout","""Shabadoo"" Black & Tan Ale",# 100,#'s Ale,#9,'t Gaverhopke Extra,'t Smisje BBBourgondier,'t Smisje Blond,'t Smisje Calva Reserva,'t Smisje Dubbel,'t Smisje Grande Reserva,'t Smisje IPA+,'t Smisje Kerst,(512) IPA,(512) Pecan Porter,(512) Whiskey Barrel Aged Double Pecan Porter,.357 Imperial Pilsner,10 Commandments,10 Degrees Below,10 Squared Fish Tale Ale,10.0,100 Meter Ale,100% Barrel Fermented Autumn Maple,1000 IBU,10th Anniversary Ale,10th Anniversary Double India Pale Ale,110K+OT Batch #2 - I.R.I.S.,110K+OT Batch #2 - I.R.I.S. - Barrel Aged,110K+OT Batch #3 - The Other West Coast IPA,110K+OT Batch #4 - Imperial Red Ale,11th Anniversary Double Wheat,11th Hour IPA,12 Dogs Of Christmas Ale,120 Minute IPA,12th Anniversary Undercover Investigation Shut-Down Ale,13 Rebels ESB,14'ER ESB,1516 Bavarian Lager,1554 Enlightened Black Ale,15th Anniversary Double Wench,15th Anniversary Wood Aged,16th Anniversary Wood Aged,17th Anniversary Wood Aged Double IPA,1800 Old English IPA,1837.0,1855 Celebration Lager,1888 Bock,18th Anniversary Pilsner,1900 Amber Lager,1919 - Choc Beer,19th Anniversary Belgian Style Golden Ale,2 Turtle Doves,200 Meter Ale,2006 Double Bastard Ale Aged In Brandy Barrels,2007 Vintage Oak Aged Ale (Barrel No. 230),2008 Vintage Oak Aged Ale (Barrel No. 96),2009 Correction Ale,2010 Christmas Ale,2012 Black Ale,20th Anniversary Pilsner,20th Anniversary Wit,21 IPA,21st Amendment IPA,21st Anniversary Belgian Strong Ale Aged With Old Vine Zin Grapes,21st Anniversary Fresh Hop Pale Ale,22nd Anniversary Vanilla Imperial Stout,25th Anniversary Vanilla Doubledog,2XIPA,2nd Anniversary Double IPA,2° Below Winter Ale,3 French Hens,3 Horses Holland Lager Beer,3 Judges Barleywine,3 Monts,3 Monts Grande Réserve Spécial Ale,30th Anniversary - Fritz And Ken's Ale,30th Anniversary - Grand Cru,30th Anniversary - Jack & Ken's Ale,"30th Anniversary- Charlie, Fred And Ken's Bock",30th Street Pale Ale,312 Urban Wheat,32/50,33 Export,3767 - Belgian IPA With Brett,3rd Anniversary Double IPA,4 Calling Birds,4 Seasons IPA,400 Pound Monkey,420 India Pale Ale,422 Pale Wheat Ale,46'er Pale Ale,471 ESB,471 IPA,483 Pale Ale,4th Anniversary Double IPA,5 A.M. Saint,5 Barrel Pale Ale,5 Day IPA,5 Years Later,5th Anniversary,5th Anniversary Ale,60 Minute IPA,60 Winks,606 Ale,6288 Stout,668 The Neighbor Of The Beast,6X,75 Minute IPA,77 Lager,8 Ball Stout,8-4-1 Expedition,80/- (Eighty Shilling),84/09 Double Alt (25th Anniversary Beer),840 India Pale Ale,8th Street Ale,90 Minute IPA,? The Riddler ?,A Little Sumpin' Extra! Ale,A Little Sumpin' Sumpin' Ale,A Little Sumpin' Wild,A Saison Darkly (Import Series Vol. #2),A-10 Warthog Double IPA,ACME California Brown Ale,ACME California IPA,ACME California Pale Ale,Aardmonnik - Earthmonk,Aass Bock,Abandon Ship Smoked Lager,Abbaye De Floreffe Double,Abbaye De Floreffe Prima Melior,Abbaye De Floreffe Triple,Abbaye De Saint Bon-Chien,Abbaye De Saint Bon-Chien Grand Cru (Aged In Trousseau Barrels),Abbaye Des Rocs Grand Cru Belgian Special Brown,Abbaye Des Rocs Spéciale Noël,Abbaye Des Rocs [Brasserie Des Rocs Brune],Abbey Ale,Abbey Belgian Style Ale,Abbey Dubbel,Abbey Six,Abbey Triple,Abbot Ale,Abner Imperial IPA,Abominable Winter Ale,Abrasive Ale,Acai Berry Wheat,Ace Of Spades,Acer Quercus - The Bruery / Lawson's Finest Liquids Collaboration,Aces & Ates,Achel Trappist Extra,Ad Astra Ale,Adam,Adam From The Wood,Adelscott,Adnams Bitter,Adnams Broadside,Adnams Broadside Strong Original,Adnams SSB (Suffolk Special Bitter),Adriaen Brouwer Dark Gold Ale,Aecht Schlenkerla Eiche,Aecht Schlenkerla Fastenbier,Aecht Schlenkerla Helles Lagerbier,Aecht Schlenkerla Rauchbier Märzen,Aecht Schlenkerla Rauchbier Urbock,Aecht Schlenkerla Rauchbier Weizen,Affligem Blond,Affligem Dubbel,Affligem Noël,Affligem Tripel,African Amber,Aftermath Imperial IPA,Afternoon Delight,Agave Wheat,Akevitt Porter (Barrel-Aged Porter),Alamo Golden Ale,Alaskan Amber,Alaskan Baltic Porter (Pilot Series),Alaskan

In [192]:
user_final_rating.iloc[0].sort_values(ascending=False)[0:5]

beer_name
Founders Breakfast Stout               589.289218
Old Rasputin Russian Imperial Stout    541.838953
Stone Ruination IPA                    526.061802
Sierra Nevada Celebration Ale          517.661141
Stone IPA (India Pale Ale)             484.142255
Name: 0110x011, dtype: float64

## Item Based using Cosine Similarity

In [194]:
beer_features = train.pivot_table( index='review_profilename', columns='beer_name', values='review_overall' ).T
beer_features.head()

review_profilename,0110x011,05Harley,0tt0,100floods,1099.0,11osixBrew,1759Girl,1Adam12,1fastz28,1morebeer,1whiskey,2BDChicago,2xHops,321jeff,32hoss32,33nickadams,3HovsepiansBrews,3Vandoo,3rdRowTailgater,4000qtrap,49degrees,4DAloveofSTOUT,4thoseabout2bock,51mmz0rz,57andcounting,57md,67cuda,7beerphil,7ways,828beagle,86MonteSS,86sportster883,99bottles,9InchNails,A380,ABCJiugui,ABHatter,ABOWMan,AChamberlain,ADR,ADZA,AEK,AJacob81,AKBelgianBeast,ALEBOY,ALESwhatCURESya,AMo,AMoceri7,ASymon13,ATLbeerDog,ATPete,AVDeuph,AWolfAtTheDoor,AaronHomoya,AaronRed,AbePhroman,Aberration,Abita45,Absumaster,AccidentalKate,AdamBear,AdamGarcia,AdamMY,Adamdc,Adamthome1,AdmiralDiaz,Admiraldinty,Adrianized,Aenema,Afterburner,AgentMunky,AgentZero,AgingWarrior,Agold,Ahhdball,Aiptasia,Airyk12,AlCaponeJunior,AlFromBayShore,AlbertF,Albinoh,AleDrinkToThat,AleWatcher,AlephNull,AlesRTasty,AlexF,AlexFields,AlexJ,AlexLMS,AlexanderSelby,Alexmc2,Alieniloquium,Alkey,AllFineThings,AllThingsNinja,Allpe45,Alpar,AltBock,Amalak,AmberMan22,Ambiorix,Amdahl,AmericanGothic,AmishT,AnPiobaire,AnalogReigns,Anaz,AndIWantYou,AndoBrew,AndreiX,Andreji,Andrew644,AndrewNations,Andrewtherooster,Andy,AndyAvalanche,Anonmatel,Another6Pack,Anthony1,Antilochus,Apellonious,Applesauce1,Arbitrator,Archemedies,Ardon,Arenbeer,Arithmeticus,Armand,ArrogantB,ArrogantDeadGuy,Asgeirr,AshevilleSippin,AshleyMiller,AtLagerHeads,Atlas1,AtlasBrew,Atricapilla,Atron67,AtrumAnimus,AudioGilz,Augustiner719,Auracom,AustinBeerLvr,Avagadro,Avantyawn,Ave,Avryle,AwYeh,Axic10,AylwinForbes,Azurescens,B967ierhunter177,BACabrera,BB1313,BBM,BBP,BBri,BCMan,BDIMike,BDJake,BDLbrewster,BDTyre,BEERME528,BEERMILER12,BEERchitect,BKotch,BMan1113VR,BMoldy,BMoney575,BS68280,BSF,BTPete,BUNZILLA,BWH3,BWMKappaSig,BZero007,BabaJohn,BabyBastard,Backer2004,BacklitAvenger,BadBadger,BadRockBeer,Badbobx,Bagman007,Baileyuk,Bajasith,BamaBeerBuff,BamaScholar,BamaViper1,Bamabrew22,Bananaman,BanditTA,BanditoB,Baraka,Barfwin,BarleyMine,BarnacleBill,BaronRogue,Barraclou,BarrelBrother,BarrelO,BarryMFBurton,BarryNL,BarrytheBear,BasementBer,BassAssassin,BassBone,Bassetdawg,Basshunter,BatDaddy,BavarianTex,BayouBengal1537,Bcant,BdubleEdubleRUN,BeRanger,BeTheBall,BeaBeerWife,BeanBone,BearJohnson,BeardedBoffin,BeardedRiker,BeardedSquash,BearsOnAcid,Beastdog75,Beaver13,BedetheVenerable,Beefeater57,BeefyMee,Beejay,BeenJammin,Beer4Baltimore,BeerAdvocate,BeerAmbassador,BeerAndBourbon,BeerAngel,BeerBeatnik,BeerBeing,BeerBelcher,BeerBob,BeerBubby,BeerBuckeye,BeerBuilder,BeerCheese,BeerCon5,BeerDawgs,BeerDocT,BeerDreadz,BeerFMAndy,BeerFan,BeerFiend2002,BeerFink,BeerForMuscle,BeerGodz,BeerHonky,BeerImmediately,BeerIsland,BeerLover48Fan,BeerLover729,BeerLover99,BeerMan7777777,BeerManDan,BeerMansGirl,BeerMark,BeerMasterD,BeerMaverick,BeerMe20,BeerMeInMS,BeerNemesis,BeerNinja007,BeerNoobNC,BeerNutta,BeerNutz324,BeerPanther,BeerPhan86,BeerResearcher,BeerRun13,BeerShark,BeerShirts,BeerSingh,BeerSki,BeerSox,BeerStine,BeerTaster,BeerThursdays,BeerTruth,BeerVikingSailor,BeerZack,Beerandraiderfan,Beerbluesbiology,Beerbuddha,Beerenauslese,BeerisfoodLLC,Beerisheaven,Beerislife652854,Beernoisseur,Beeromaniac,Beersignal,Beertracker,Beervis,Beerwolf17,Beeryce,Beerzebub,BehindtheFinBrew,BelgianAleSlut,BelgianBeers,BelgiumBill,Ben4881,BenConnery,BenZ,Bendurgin,Bennettrox,BennyL1,BerkeleyBeerMan,Berthold,Bestuurder,BetterDarker,BevMoAndrew,Bfarr,BgThang,BierFan,BierManVA,BierReise,BierStein711,Bierbauch,BiereBlanche,BieresDesMoines,Bierguy5,Bierlerner,Bierman9,Bierschenker,Biffster,Big78CJ,BigAl723,BigB415,BigBigPickle,BigBlacke,BigBry,BigCheese,BigDRS,BigDaddyWil,BigDank,BigHead,BigIrishman,BigJTrucker,BigJim5021,BigMcLargeHuge,BigMike,BigPlay1824,BigRedN,BigWilly,Bigfatbino,Biggness,Bighuge,Bigswig,BilbosNuts,BillHilly,Billolick,BillyB,BillyBoysPie,BillyBrew,Birchgrove77,BirdFlu,Birdiebeagle,Birmingham,Biscuit033,Bisho,BittRnessabounds,BitteBier,BitterBrian,BitterPachyderm,Bitterbill,Bitterman,Bizz88,BlackAngus,BlackBearForge,BlackHaddock,BlackJeremiah,Blakaeris,Blanco,Blast

In [196]:
from sklearn.metrics.pairwise import pairwise_distances

item_correlation = 1 - pairwise_distances(beer_features.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

[[1.         0.         0.04656427 ... 0.04744468 0.12137516 0.07372924]
 [0.         1.         0.0096709  ... 0.05376892 0.00879036 0.07595883]
 [0.04656427 0.0096709  1.         ... 0.17272784 0.09776195 0.15992855]
 ...
 [0.04744468 0.05376892 0.17272784 ... 1.         0.12974331 0.19892819]
 [0.12137516 0.00879036 0.09776195 ... 0.12974331 1.         0.12987641]
 [0.07372924 0.07595883 0.15992855 ... 0.19892819 0.12987641 1.        ]]


In [197]:
item_predicted_ratings = np.dot((beer_features.fillna(0).T),item_correlation)
item_predicted_ratings

array([[26.61319596, 20.19008295, 55.50286423, ..., 85.08812264,
        41.10220381, 58.85062094],
       [19.13920092, 22.29266312, 44.95717547, ..., 60.05993083,
        34.23851779, 51.17842568],
       [ 7.57707575,  7.48185361, 13.58837404, ..., 16.11849071,
        14.87722096, 16.78970082],
       ...,
       [ 9.88104902,  9.23192451, 14.24915522, ..., 18.41006072,
        18.57442396, 17.47477724],
       [12.38140798, 11.52410778, 17.64236875, ..., 21.13029392,
        22.13088179, 18.96265476],
       [11.70502216, 13.29618753, 18.1278956 , ..., 21.56057541,
        25.34198289, 23.96930094]])

In [198]:
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

beer_name,"""Old Yeltsin"" Imperial Stout","""Shabadoo"" Black & Tan Ale",# 100,#'s Ale,#9,'t Gaverhopke Extra,'t Smisje BBBourgondier,'t Smisje Blond,'t Smisje Calva Reserva,'t Smisje Dubbel,'t Smisje Grande Reserva,'t Smisje IPA+,'t Smisje Kerst,(512) IPA,(512) Pecan Porter,(512) Whiskey Barrel Aged Double Pecan Porter,.357 Imperial Pilsner,10 Commandments,10 Degrees Below,10 Squared Fish Tale Ale,10.0,100 Meter Ale,100% Barrel Fermented Autumn Maple,1000 IBU,10th Anniversary Ale,10th Anniversary Double India Pale Ale,110K+OT Batch #2 - I.R.I.S.,110K+OT Batch #2 - I.R.I.S. - Barrel Aged,110K+OT Batch #3 - The Other West Coast IPA,110K+OT Batch #4 - Imperial Red Ale,11th Anniversary Double Wheat,11th Hour IPA,12 Dogs Of Christmas Ale,120 Minute IPA,12th Anniversary Undercover Investigation Shut-Down Ale,13 Rebels ESB,14'ER ESB,1516 Bavarian Lager,1554 Enlightened Black Ale,15th Anniversary Double Wench,15th Anniversary Wood Aged,16th Anniversary Wood Aged,17th Anniversary Wood Aged Double IPA,1800 Old English IPA,1837.0,1855 Celebration Lager,1888 Bock,18th Anniversary Pilsner,1900 Amber Lager,1919 - Choc Beer,19th Anniversary Belgian Style Golden Ale,2 Turtle Doves,200 Meter Ale,2006 Double Bastard Ale Aged In Brandy Barrels,2007 Vintage Oak Aged Ale (Barrel No. 230),2008 Vintage Oak Aged Ale (Barrel No. 96),2009 Correction Ale,2010 Christmas Ale,2012 Black Ale,20th Anniversary Pilsner,20th Anniversary Wit,21 IPA,21st Amendment IPA,21st Anniversary Belgian Strong Ale Aged With Old Vine Zin Grapes,21st Anniversary Fresh Hop Pale Ale,22nd Anniversary Vanilla Imperial Stout,25th Anniversary Vanilla Doubledog,2XIPA,2nd Anniversary Double IPA,2° Below Winter Ale,3 French Hens,3 Horses Holland Lager Beer,3 Judges Barleywine,3 Monts,3 Monts Grande Réserve Spécial Ale,30th Anniversary - Fritz And Ken's Ale,30th Anniversary - Grand Cru,30th Anniversary - Jack & Ken's Ale,"30th Anniversary- Charlie, Fred And Ken's Bock",30th Street Pale Ale,312 Urban Wheat,32/50,33 Export,3767 - Belgian IPA With Brett,3rd Anniversary Double IPA,4 Calling Birds,4 Seasons IPA,400 Pound Monkey,420 India Pale Ale,422 Pale Wheat Ale,46'er Pale Ale,471 ESB,471 IPA,483 Pale Ale,4th Anniversary Double IPA,5 A.M. Saint,5 Barrel Pale Ale,5 Day IPA,5 Years Later,5th Anniversary,5th Anniversary Ale,60 Minute IPA,60 Winks,606 Ale,6288 Stout,668 The Neighbor Of The Beast,6X,75 Minute IPA,77 Lager,8 Ball Stout,8-4-1 Expedition,80/- (Eighty Shilling),84/09 Double Alt (25th Anniversary Beer),840 India Pale Ale,8th Street Ale,90 Minute IPA,? The Riddler ?,A Little Sumpin' Extra! Ale,A Little Sumpin' Sumpin' Ale,A Little Sumpin' Wild,A Saison Darkly (Import Series Vol. #2),A-10 Warthog Double IPA,ACME California Brown Ale,ACME California IPA,ACME California Pale Ale,Aardmonnik - Earthmonk,Aass Bock,Abandon Ship Smoked Lager,Abbaye De Floreffe Double,Abbaye De Floreffe Prima Melior,Abbaye De Floreffe Triple,Abbaye De Saint Bon-Chien,Abbaye De Saint Bon-Chien Grand Cru (Aged In Trousseau Barrels),Abbaye Des Rocs Grand Cru Belgian Special Brown,Abbaye Des Rocs Spéciale Noël,Abbaye Des Rocs [Brasserie Des Rocs Brune],Abbey Ale,Abbey Belgian Style Ale,Abbey Dubbel,Abbey Six,Abbey Triple,Abbot Ale,Abner Imperial IPA,Abominable Winter Ale,Abrasive Ale,Acai Berry Wheat,Ace Of Spades,Acer Quercus - The Bruery / Lawson's Finest Liquids Collaboration,Aces & Ates,Achel Trappist Extra,Ad Astra Ale,Adam,Adam From The Wood,Adelscott,Adnams Bitter,Adnams Broadside,Adnams Broadside Strong Original,Adnams SSB (Suffolk Special Bitter),Adriaen Brouwer Dark Gold Ale,Aecht Schlenkerla Eiche,Aecht Schlenkerla Fastenbier,Aecht Schlenkerla Helles Lagerbier,Aecht Schlenkerla Rauchbier Märzen,Aecht Schlenkerla Rauchbier Urbock,Aecht Schlenkerla Rauchbier Weizen,Affligem Blond,Affligem Dubbel,Affligem Noël,Affligem Tripel,African Amber,Aftermath Imperial IPA,Afternoon Delight,Agave Wheat,Akevitt Porter (Barrel-Aged Porter),Alamo Golden Ale,Alaskan Amber,Alaskan Baltic Porter (Pilot Series),Alaskan

In [199]:
item_final_rating.iloc[0].sort_values(ascending=False)[0:5]

beer_name
AleSmith Speedway Stout     100.062211
The Abyss                    96.011490
Imperial Stout               95.585053
Founders Breakfast Stout     95.510032
AleSmith IPA                 93.634619
Name: 0110x011, dtype: float64